In [ ]:
import os
import json
import requests
import time
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from dotenv import load_dotenv
from openai import AzureOpenAI

# 환경변수 로드
load_dotenv()

# ============================================
# 1. 유틸리티 함수: 대화 로그 프린트 및 GPT 프롬프트 생성
# ============================================
def print_pretty_messages(messages):
    print("\n📨 메시지 대화 로그\n" + "-" * 40)
    for message in reversed(messages.data):  # 최신순으로 출력
        role = "🧑 사용자" if message.role == "user" else "🤖 GPT"
        print(f"\n[{role}]")
        for block in message.content:
            if block.type == "text":
                print(block.text.value)
        print("-" * 40)

def make_prediction_prompt(genre, seat, region, marketing, star_power, audience_pred, weekday_rate, weekend_rate):
    return f"""
감사합니다. 과거 유사 뮤지컬 데이터를 참고하고, 

Random Forest Regressor 모델을 적용해 예측해보겠습니다.

잠시만 기다려주세요…

▶ 장르: {genre}
▶ 지역: {region}
▶ 좌석 수: {seat:,}석
▶ 마케팅 예산: {marketing:,}만 원
▶ 스타 파워: {star_power}점

이에 기반한 예측 모델(Random Forest Regressor)을 적용한 결과:

▶ 예상 관객 수는 약 {audience_pred:,}명 (±500명) 정도로 추정됩니다.
▶ 평일 평균 점유율은 {weekday_rate}%,
   주말 평균 점유율은 {weekend_rate}%로 예상됩니다.

(대시보드 링크 안내)
대시보드에서 일자별 누적 관객 추이 그래프와 유사 공연 대비 비교 바차트를 보실 수 있습니다.

다른 정보도 궁금하신가요? 예를 들어 손익분기점, ROI 분석도 가능합니다.
"""

# ============================================
# 2. 랜덤포레스트 모델 학습 (더미 데이터 기반)
# ============================================
np.random.seed(42)
dummy_data = pd.DataFrame({
    'seat': np.random.randint(300, 2000, size=50),
    'marketing': np.random.randint(1000, 5000, size=50),
    'star_power': np.random.randint(1, 6, size=50)
})
dummy_target = (
    dummy_data['seat'] * 10 +
    dummy_data['marketing'] * 1.5 +
    dummy_data['star_power'] * 800 +
    np.random.normal(0, 1000, size=50)
)
rf_model = RandomForestRegressor(n_estimators=10, random_state=42)
rf_model.fit(dummy_data, dummy_target)

def predict_audience_rf(seat, marketing, star_power): # 좌석 , 예산 , 스타파워 입력
    """
    입력된 좌석 수, 마케팅 예산(만원), 스타 파워(1~5점)를 기반으로
    예상 관객 수를 예측합니다.
    추가로, 평일 점유율은 60%, 주말 점유율은 85%로 고정하여 반환합니다.
    (실제 환경에서는 이 부분을 더 정교한 모델로 대체할 수 있습니다.)
    """
    input_data = pd.DataFrame([{
        'seat': seat,
        'marketing': marketing,
        'star_power': star_power
    }])
    pred = rf_model.predict(input_data)[0]
    # 예시: 실제 예측값이 약 8500명 정도 나오도록 (±500명 오차 범위)
    # 그리고 평일 60%, 주말 85%로 고정
    weekday_rate = 60
    weekend_rate = 85
    return int(pred), weekday_rate, weekend_rate

# ============================================
# 3. CLU 호출: Azure CLU API에서 사용자 발화를 분석
# ============================================
def call_clu_api(text):
    clu_endpoint_base = os.getenv("clu_endpoint")  # 예: https://language5team.cognitiveservices.azure.com
    clu_api_key = os.getenv("clu_key")
    clu_id = os.getenv("clu_id")
    project_name = "PPCLU"
    deployment_name = "ppv1-test"
    headers = {
        "Ocp-Apim-Subscription-Key": clu_api_key,
        "Content-Type": "application/json",
        "Apim-Request-Id": clu_id
    }
    url = clu_endpoint_base
    body = {
        "kind": "Conversation",
        "analysisInput": {
            "conversationItem": {
                "id": "1",
                "participantId": "user",
                "text": text,
                "modality": "text",
                "language": "ko"
            }
        },
        "parameters": {
            "projectName": project_name,
            "deploymentName": deployment_name,
            "verbose": True,
            "stringIndexType": "TextElement_V8"
        }
    }
    response = requests.post(url, headers=headers, json=body)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"CLU API 호출 실패: {response.status_code}")
        print(response.text)
        return None

# (rag_agent_utils.py에 정의된 함수들을 import)
from rag_agent_utils import parse_clu_response, build_kopis_query, make_gpt_prompt

# ============================================
# 4. Azure OpenAI GPT 호출 및 스레드 실행 (RAG)
# ============================================
client = AzureOpenAI(
    azure_endpoint=os.getenv("oai_assistant_endpoint"),
    api_key=os.getenv("oai_assistant_key"),
    api_version="2024-05-01-preview"
)

assistant = client.beta.assistants.create(
    model="gpt-4o",  # 모델 배포 이름 (예: gpt-4o)
    instructions="""너는 한국의 공연 정보를 잘 알고 있는 똑똑한 문화예술 전문가 도우미야.
사용자가 질문하면, 벡터 저장소에서 관련 정보를 찾아서 친절하고 정확하게 알려줘.
응답은 정중한 말투를 사용하며, 공연/예매정보/상세정보/통계 등을 포함해 요약해줘.""",
    tools=[{"type": "file_search"}, {"type": "code_interpreter"}],
    tool_resources={"file_search": {"vector_store_ids": ["vs_rk9EOjktnff6RYK12Hx5jn7J"]}, "code_interpreter": {"file_ids": []}},
    temperature=0.3,
    top_p=0.95
)

def request_gpt(prompt):
    # Azure OpenAI GPT 호출용 URL (전체 URL)
    azure_oai_endpoint = os.getenv("oai_endpoint") 
    azure_oai_key = os.getenv("oai_key")
    headers = {
        "Content-Type": "application/json",
        "api-key": azure_oai_key
    }
    body = {
        "messages": [
            {
                "role": "system",
                "content": "공연 산업에 대한 지식이 아주 해박하여, 관련 질문에 대해 자세히 답변할 수 있는 전문가 역할을 수행해 주세요."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 1000
    }
    response = requests.post(azure_oai_endpoint, headers=headers, json=body)
    if response.status_code == 200:
        response_json = response.json()
        content_text = response_json['choices'][0]['message']['content']
        citations = response_json['choices'][0]['message'].get('context', {}).get('citations', [])
        return content_text, citations
    else:
        return response.status_code, response.text

# ============================================
# 5. 메인 통합 파이프라인 실행
# ============================================
def main():
    # 사용자 발화 예시 (관객 예측 요청)
    user_query = "이번에 새로 준비 중인 뮤지컬의 관객 수를 예측하고 싶어요."
    print("질문:", user_query)

    # CLU API 호출 → 엔터티 추출
    clu_resp = call_clu_api(user_query)
    if not clu_resp:
        return
    entities = parse_clu_response(clu_resp)
    print("\n추출된 엔터티:", entities)

    # 여기서는 사용자로부터 추가 정보를 수집했다고 가정 (좌석, 마케팅 예산, 스타파워)
    seat = 1000          # 좌석 수
    marketing = 2000     # 마케팅 예산 (만원)
    star_power = 5       # 스타 파워 (1~5점)

    # 관객 예측 (RandomForest 모델)
    audience_pred, weekday_rate, weekend_rate = predict_audience_rf(seat, marketing, star_power)
    print(f"\n예상 관객 수: {audience_pred:,}명")
    print(f"평일 평균 점유율: {weekday_rate}%, 주말 평균 점유율: {weekend_rate}%")

    # KOPIS 검색 조건 생성 (이미 CLU에서 장르, 지역, 일정 등이 추출되었다고 가정)
    kopis_query = build_kopis_query(entities)
    print("\n생성된 KOPIS 검색 조건:", kopis_query)

    # (더미) KOPIS API 호출 → 실제 공연 결과 (여기서는 더미 문자열 사용)
    def call_kopis_api(kopis_query):
        return (f"- 더미 공연 결과: {kopis_query.get('genre', '공연')} 공연 in {kopis_query.get('region', '지역')} "
                f"from {kopis_query.get('startDate', '날짜')} to {kopis_query.get('endDate', '날짜')}.")
    kopis_results = call_kopis_api(kopis_query)
    print("\nKOPIS API 검색 결과:", kopis_results)

    # GPT 프롬프트 생성 (예측 결과와 KOPIS 검색 결과 포함)
    gpt_prompt = make_prediction_prompt(
        genre=entities.get("장르", "뮤지컬"),
        seat=seat,
        region=entities.get("지역", "서울"),
        marketing=marketing,
        star_power=star_power,
        audience_pred=audience_pred,
        weekday_rate=weekday_rate,
        weekend_rate=weekend_rate
    )
    print("\n생성된 GPT 프롬프트:\n", gpt_prompt)

    # GPT 호출하여 최종 응답 생성
    gpt_response, citations = request_gpt(gpt_prompt)
    print("\n🧠 GPT 응답:\n", gpt_response)
    print("\n📚 출처:\n", citations)

    # Azure OpenAI 챗봇 스레드 실행 (실제 대화)
    thread = client.beta.threads.create()
    user_input = input("\n대화 입력 (내용을 입력하세요): ")
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input
    )
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )
    while run.status in ['queued', 'in_progress', 'cancelling']:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
    if run.status == 'completed':
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        print_pretty_messages(messages)
    elif run.status == 'requires_action':
        print("추가 작업 필요")
    else:
        print("스레드 상태:", run.status)

if __name__ == "__main__":
    main()
